# Validación de Sensor BME680 vs Estación Meteorológica UNAM

Este notebook presenta el análisis comparativo entre los datos recolectados por un sensor **BME680** (prototipo IoT) y una estación meteorológica de referencia (**UNAM**).

> **Objetivo:** Evaluar la precisión y exactitud del sensor BME680 para aplicaciones de agricultura de precisión.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configuración de estilo
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

## 1. Carga de Datos
Cargamos el archivo `comparacion_detallada.csv` que contiene los datos alineados temporalmente.

In [ ]:
# Cargar dataset
df = pd.read_csv('comparacion_detallada.csv')

# Asegurar que la columna temporal sea datetime
df['Time_Key'] = pd.to_datetime(df['Time_Key'])

# Mostrar primeras filas
df.head()

## 2. Métricas Globales
Resumen estadístico del desempeño del sensor.

In [ ]:
# Extraer métricas globales (repetidas en cada fila, tomamos la primera)
mape = df['Global_MAPE'].iloc[0]
pearson = df['Global_Pearson'].iloc[0]
rmse = df['Global_RMSE'].iloc[0]
r2 = df['Global_R2'].iloc[0]
offset = df['Diff_Temp'].mean()

print(f"=== RESULTADOS DE VALIDACIÓN ===")
print(f"Muestras Totales: {len(df)}")
print(f"MAPE (Error %):   {mape:.2f} %")
print(f"Pearson (r):      {pearson:.4f}")
print(f"RMSE (Error abs): {rmse:.2f} °C")
print(f"R² (Ajuste):      {r2:.4f}")
print(f"Offset Medio:     {offset:.2f} °C")

## 3. Análisis Visual

### 3.1 Serie de Tiempo: Comparación Directa
Observamos cómo el sensor sigue la tendencia de la estación de referencia a lo largo del tiempo.

In [ ]:
plt.figure(figsize=(14, 6))
sns.lineplot(data=df, x='Time_Key', y='Temp_UNAM', label='Referencia (UNAM)', color='black', alpha=0.7)
sns.lineplot(data=df, x='Time_Key', y='Temp_BME', label='Sensor (BME680)', color='red', alpha=0.7)

plt.title('Comparación Temporal: Temperatura BME680 vs UNAM')
plt.ylabel('Temperatura (°C)')
plt.xlabel('Fecha / Hora')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 3.2 Diagrama de Dispersión (Correlación)
Evaluamos la linealidad entre ambas mediciones. Una línea recta ideal a 45 grados indicaría concordancia perfecta.

In [ ]:
plt.figure(figsize=(8, 8))
sns.scatterplot(data=df, x='Temp_UNAM', y='Temp_BME', alpha=0.6)

# Línea de identidad (Ideal)
lims = [0, 30]
plt.plot(lims, lims, '--r', label='Identidad (Ideal)')

plt.title(f'Correlación de Pearson: {pearson:.4f}')
plt.xlabel('Referencia UNAM (°C)')
plt.ylabel('Sensor BME680 (°C)')
plt.legend()
plt.grid(True)
plt.show()

### 3.3 Distribución del Error
Analizamos el sesgo del sensor. ¿Mide sistemáticamente por encima o por debajo?

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Diff_Temp', kde=True, bins=30, color='purple')

plt.axvline(x=offset, color='red', linestyle='--', label=f'Media: {offset:.2f} °C')
plt.title('Distribución del Error (BME680 - UNAM)')
plt.xlabel('Diferencia de Temperatura (°C)')
plt.ylabel('Frecuencia')
plt.legend()
plt.show()

## 4. Conclusión
El análisis muestra una fuerte correlación lineal, validando la capacidad del sensor para seguir tendencias. El error sistemático (offset) observado sugiere la necesidad de una calibración de punto cero para corregir la exactitud absoluta.